<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

В этом задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Импортируем нужные библиотеки:

In [2]:
import math
import random
import string
import re
from string import ascii_letters
from IPython.display import clear_output
#from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, random_split
from torch.utils.data import DataLoader

In [3]:
SEED = 42
RANDOM_STATE = SEED
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True


train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')  
device

CUDA is not available.  Training on CPU ...


device(type='cpu')

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", 
                   encoding="latin", 
                   header=None, 
                   names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [44]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [72]:
examples = data["text"].sample(10)
print("\n".join(examples))

@LBOI  Wow, thats just crazy. 
@SinnerOfficial hello u! i do have a hobby! tweet tweet. 
@BimmerTech I dunno, but when you heading home 
HAPPY BDAY @Mattkean ! wish you all the best 
today might be the longest day I've yet to experience at work. 11-8. 
Trying to get train to London to present to CEO this am and none of the parking machines are accepting cards. Must be Monday. 
It's 710am i'm leaving my baby's  my car and my love  *~Goober Joe~*
@natashaturnbull &quot; fuck this  &quot; whats up? i want to have a 'drunken' chat with you. i'm only tipsy so i know what i;m saying. its hard to
@sojbagley haha, yea ... i decided to go check it, but it was a waste, it hasn't come yet. 
@QueenBxoxo u reply to meeee :$ i love yooou  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

#### Несколько функций для подготовки текстовых данных

In [5]:
def multi_letter(text: str = 'Добавьте в функцию текст', repeat: int = 1) -> str:
    '''
    Example: 'AaaaBBBBBwwwWWW 2222qqq!!!' -> 'aabbww 2222qq!!!' (if repeat = 2)
    text - any text (type:str)
    repeat - number of identical letters in one word, following each other without skipping
    '''
    r = [None]
    count = 0
    for c in text.lower():
        if c != r[-1] and c.isalpha():
            r.append(c)
            count = 0
        elif c == r[-1] and count < repeat - 1 and c.isalpha():
            r.append(c)
            count += 1
        elif not c.isalpha():
            r.append(c)
            
    return ''.join(r[1:]).strip()

In [6]:
def del_nick_name(text: str = '@NickName') -> str:
    '''
    Remove all nickname's look like @NickName or @21213_Alex! from text
    '''
    return re.sub(r'@\S+', '', text).strip()

In [7]:
def del_url(text: str) -> str:
    '''
    Remove all URL look like http://site.gov or https://site.com from text
    '''
    return re.sub(r'http\S+', '', text).strip()

In [73]:
%%time
from tqdm import tqdm
clear_row_list = []

for idx in tqdm(range(data.shape[0])):
    clear_row = data['text'][idx].lower()                                   # переводим слова в нижний регистр
    clear_row = del_url(clear_row)                                          # удалим все гипер-ссылки
    clear_row = del_nick_name(clear_row)                                    # удалим все ник-неймы
    clear_row = multi_letter(clear_row, 2)                                  # удалим повторяющиеся буквы с частотой > n
    clear_row = ''.join(l for l in clear_row \
                        if l in set(ascii_letters + \
                                    ' ')) # оставляем только буквы, пробелы
    clear_row = " ".join(clear_row.split())                                 # удалим множественные пробелы
    clear_row_list.append(clear_row)
data['text'] = clear_row_list

if sum(list(data.isnull().sum())) == 0:
    print('\033[1m' + 'NaN-строки отсутствуют')
else:
    print('\033[1m' + 'NaN-строки присутствуют в количестве:'+'\033[0m')
    print(data.isnull().sum())

100%|██████████| 1600000/1600000 [03:44<00:00, 7111.44it/s]


NaN-строки отсутствуют
CPU times: user 3min 40s, sys: 3.9 s, total: 3min 44s
Wall time: 3min 45s


In [74]:
data.to_csv('clear_data.csv')

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data = pd.read_csv("clear_data.csv", index_col=0)
data.dropna(subset=['text'],inplace=True)
data.reset_index(drop=True, inplace=True)

---

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [9]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [10]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

the confederate flag is an interesting choice to fly kinda makes me nervous


In [11]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

confederate flag interesting choice kinda makes nervous


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [12]:
word2vec = api.load("word2vec-google-news-300")

In [13]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [14]:
mean = np.mean(word2vec.vectors, axis=0)
std = np.std(word2vec.vectors, axis=0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [15]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их (нормализуй) 
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [16]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предложении (которые остались после токенизации и удаления коротких слов, конечно). 

In [17]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1278


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [91]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components=2)
#examples["transformed_features"] = pca.fit_transform(examples["features"])  # Обучим PCA на эмбеддингах слов

In [18]:
%%time
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_jobs=-1)
examples["transformed_features"] = tsne.fit_transform(examples["features"])  # Обучим TSNE на эмбеддингах слов

CPU times: user 52.9 s, sys: 1.57 s, total: 54.5 s
Wall time: 7.4 s


In [19]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [20]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1004', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [19]:
batch_size = 1024
# num_workers = 4


def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch])

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          collate_fn=average_emb)

valid_loader = DataLoader(valid,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False,
                          collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [20]:
def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    running_loss = 0
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        
        optimizer.zero_grad()

        # предсказания модели
        outp = model(features)
        
        loss = criterion(outp, targets) # лосс
        running_loss += loss.item()

        # обновление параметров модели
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {running_loss/len(train_loader):.4}")

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # предсказания модели
            outp = model(features)
            
            loss = criterion(outp, targets) # лосс
            acc = accuracy_score(targets.cpu(), outp.cpu() > 0.5) # точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [21]:
# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 10

class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH, nY):        
        super(TwoLayersNet, self).__init__()
        
        self.fc1 = nn.Sequential(
            nn.Linear(nX, nH),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(nH, nY),
            nn.Sigmoid(),
            nn.Flatten(start_dim=0)
        )
        
    def forward(self, x):                        
        x = self.fc1(x)                                            
        x = self.fc2(x)                                           
        return x

model = TwoLayersNet(vector_size, vector_size, 1) # модель

model = model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

In [22]:
from torchsummary import summary
summary(model,(vector_size,vector_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 300, 300]          90,300
              ReLU-2             [-1, 300, 300]               0
            Linear-3               [-1, 300, 1]             301
           Sigmoid-4               [-1, 300, 1]               0
           Flatten-5                       [-1]               0
Total params: 90,601
Trainable params: 90,601
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.34
Forward/backward pass size (MB): 1.38
Params size (MB): 0.35
Estimated Total Size (MB): 2.07
----------------------------------------------------------------


Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [99]:
from tqdm.notebook import tqdm

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.49269072818756104, 'Test Acc': 0.7615462150399543}


Epoch 2. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.48324637830257416, 'Test Acc': 0.7709069099600457}


Epoch 10. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.4830189974308014, 'Test Acc': 0.7709851419805936}


In [100]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size,
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/312 [00:00<?, ?it/s]

{'Test Loss': 0.48220435787851995, 'Test Acc': 0.7705194204251086}


Точность классификации твитов на двухслойной нейронной модели составила 0.77, что является достаточно хорошим результатом для такого упрощенного решения. Для задачи бинарной классификации точность важнее, чем итоговое значение функции потерь.

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

#### Вариант 1

Создадим новый класс датасета с учётом контекста. Контекстом будут являться слова по 3 шт справа и слева от нашего неизвестного.

In [23]:
class TwitterDataset_context(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line
    
    def get_embeddings_(self, tokens):
        
        embeddings = []
        
        for idx, token in enumerate(tokens):
            if token in self.word2vec:
                embeddings.append(self.word2vec.get_vector(token))
            else:
                context = tokens[max(idx - 3, 0) : min(idx + 3, len(tokens))]
                context.remove(token)
                context_embedding = np.sum(np.array([(self.word2vec.get_vector(w) - self.mean) 
                                                     / self.std for w in context if w in self.word2vec]), axis=0)
                if context_embedding.all() == 0:
                    continue
                embeddings.append(context_embedding)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [24]:
dev_context = TwitterDataset_context(dev_data, "text", "emotion", word2vec)

Подготовим загрузчики данных.

In [25]:
train_size = math.ceil(len(dev_context) * 0.8)

train, valid = random_split(dev_context, [train_size, len(dev_context) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Создадим модель, оптимизатор и целевую функцию.

In [26]:
vector_size = dev_context.word2vec.vector_size
lr = 1e-2
num_epochs = 10

model_context = TwoLayersNet(vector_size, vector_size, 1)

model_context = model_context.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_context.parameters(), lr=lr, betas=(0.9, 0.999))

Обучим модель и протестируем её.

In [105]:
best_metric = np.inf
for e in range(num_epochs):
    training(model_context, optimizer, criterion, train_loader, e, device)
    log = testing(model_context, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model_context.state_dict(), "model_context.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.522581730723381, 'Test Acc': 0.7411192744006849}


Epoch 2. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5160181899070739, 'Test Acc': 0.7515762521404109}


Epoch 10. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5139585673809052, 'Test Acc': 0.7511023829908676}


In [106]:
test_loader = DataLoader(
    TwitterDataset_context(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size,
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model_context.load_state_dict(torch.load("model_context.pt", map_location=device))

print(testing(model_context, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/312 [00:00<?, ?it/s]

{'Test Loss': 0.5099696484513772, 'Test Acc': 0.751566906178154}


###### Суммирование эмбеддингов для неизвестных слов по их ближайшему контексту не улучшило точность классификации при прочих равных условиях.

#### Вариант 2

Создадим новый класс датасета с токенизацией, без знаков пунктуации и отбросим все слова, состоящие менее чем из 3 букв.

In [27]:
from tqdm import tqdm
tokenized_data = []
tokenizer = nltk.WordPunctTokenizer()

for i in tqdm(range(len(data))):
    line = tokenizer.tokenize(data["text"][i].lower())
    filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
    filtered_string = " ".join(filtered_line)
    tokenized_data.append(filtered_string)

100%|██████████| 1596337/1596337 [00:35<00:00, 44741.65it/s]


Полученные данные

In [29]:
for i in range(5):
    print(tokenized_data[i])

aww thats bummer you shoulda got david carr third day
upset that cant update his facebook texting and might cry result school today also blah
dived many times for the ball managed save the rest out bounds
whole body feels itchy and like its fire
its not behaving all mad why here because cant see you all over there


Обучим TfidfVectorizer на полном датасете

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(lowercase=False)
vectors = vectorizer.fit_transform(data['text'].tolist())
vectors.shape

(1596337, 400769)

SVD преобразование для уменьшения размерности

In [31]:
from sklearn.decomposition import TruncatedSVD
 
trun_svd = TruncatedSVD(n_components=300)
vectors_transformed = trun_svd.fit_transform(vectors)
vectors_transformed.shape

(1596337, 300)

Для каждого слова текста получим его эмбеддинг из Tfidf с помощью TfidfVectorizer из sklearn. Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf.

In [32]:
class TwitterDataset_tfidf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, 
                 word2vec: gensim.models.Word2Vec, tfidf, t_vectors: np.array):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf = tfidf
        self.t_vectors = t_vectors

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        self.mean_tfidf = np.mean(t_vectors, axis=0)
        self.std_tfidf = np.std(t_vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line
    
    def get_w2v_vector_(self, token):
        return (self.word2vec.get_vector(token) - self.mean) / self.std
    
    def get_tfidf_vector_(self, token):
        tfidf_idx = self.tfidf.vocabulary_[token]
        tfidf_vector = self.t_vectors[tfidf_idx]
        return (tfidf_vector - self.mean_tfidf) / self.std_tfidf

    def get_embeddings_(self, tokens):
        embeddings = []
        
        for token in tokens:
            if token in self.word2vec:
                w2v_emb = self.get_w2v_vector_(token)
                tfidf_emb = self.get_tfidf_vector_(token)
                embeddings.append(w2v_emb + tfidf_emb)
            else:
                tfidf_emb = self.get_tfidf_vector_(token)
                embeddings.append(tfidf_emb)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [33]:
dev_tfidf = TwitterDataset_tfidf(dev_data, "text", "emotion", word2vec, vectorizer, vectors_transformed)

Подготовим загрузчики данных.

In [34]:
train_size = math.ceil(len(dev_tfidf) * 0.8)

train, valid = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Создадим модель, оптимизатор и целевую функцию.

In [36]:
vector_size = dev_tfidf.word2vec.vector_size
lr = 1e-2
num_epochs = 10

model_tfidf = TwoLayersNet(vector_size, vector_size, 1)

model_tfidf = model_tfidf.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_tfidf.parameters(), lr=lr, betas=(0.9, 0.999))

Обучим модель и протестируем её.

In [38]:
from tqdm.notebook import tqdm
best_metric = np.inf
for e in range(num_epochs):
    training(model_tfidf, optimizer, criterion, train_loader, e, device)
    log = testing(model_tfidf, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model_tfidf.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5058207991123199, 'Test Acc': 0.7514604380707762}


Epoch 2. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5041352052688599, 'Test Acc': 0.7685256314212329}


Epoch 10. Train Loss: 0:   0%|          | 0/997 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5086508734226227, 'Test Acc': 0.7666794021118721}


In [39]:
test_loader = DataLoader(
    TwitterDataset_tfidf(test_data, "text", "emotion", word2vec, vectorizer, vectors_transformed), 
    batch_size=batch_size,  
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model_tfidf.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model_tfidf, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/312 [00:00<?, ?it/s]

{'Test Loss': 0.4882177080099399, 'Test Acc': 0.7658567141423788}


###### Вариант суммирования Tfidf эмбеддингов с предобученными word2vec оказался лучше, чем вариант суммирования эмбедингов по контесту для неизвестных слов, и лишь немного уступает нашему первоначальному варианту по точности.